# 09. Enhanced Prophet v2: 2023년 실패 보완

---

## 개선사항 (v1 대비)

### 1. 거시경제 피처 추가
- **DGS10** (10년 금리): level, change_20d, zscore_252
- **WTI** (원유 가격): log, mom_3M, change_20d
- **효과**: 금리 급등/유가 변동에 따른 섹터 영향 반영

### 2. 레짐 피처 추가
- **Market_Vol**: 시장 전체 변동성
- **CrossSection_Dispersion**: 섹터 간 수익률 분산
- **효과**: 2023년 같은 레짐 전환 시기 탐지

### 3. 단기 모멘텀 강화
- **Mom_1M**: 1개월 수익률
- **Mom_6M**: 6개월 수익률
- **Mom_Accel**: 가속도 (1M - 6M)
- **효과**: 2023 Tech 급반등 포착

### 4. Prophet cps 동적 조정
- 섹터 변동성 기반 자동 조정
- high vol (>0.40): cps=0.10
- mid vol (0.30~0.40): cps=0.05
- low vol (<0.30): cps=0.03

### 5. XGBoost Early Stopping
- train/val split (80/20)
- early_stopping_rounds=20
- **효과**: Energy 과적합 방지

---

## Section 1. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

from prophet import Prophet
import xgboost as xgb
import holidays

from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

# 개선된 피처 모듈 import
from sector_rotation_features import (
    load_and_prepare_macro,
    merge_macro_to_sector_panel,
    add_regime_features,
    add_momentum_features,
    calculate_dynamic_cps
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# 한글 폰트
import koreanize_matplotlib

In [ ]:
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Data_set'
OUTPUT_DIR = DATA_DIR / 'Enhanced_Prophet_v2_Results'
TABLEAU_DIR = DATA_DIR / 'Tableau_Csv'

OUTPUT_DIR.mkdir(exist_ok=True)

print(f"프로젝트 경로: {PROJECT_ROOT}")
print(f"결과 저장: {OUTPUT_DIR}")

---

## Section 2. 데이터 로드

In [ ]:
print("=" * 70)
print("주식 데이터 로드")
print("=" * 70)

df = pd.read_csv(DATA_DIR / 'stock_features_clean.csv')
df['Date'] = pd.to_datetime(df['Date'])

df = df.dropna(subset=['Date', 'Close', 'Sector', 'Daily_Return'])
df = df[df['Sector'] != 'Unknown']

print(f"\n 로드 완료")
print(f"  Shape: {df.shape}")
print(f"  기간: {df['Date'].min().date()} ~ {df['Date'].max().date()}")
print(f"  기업 수: {df['Company'].nunique()}")
print(f"  섹터 수: {df['Sector'].nunique()}")

### 거시경제 데이터 로드 (개선사항 #1)

In [ ]:
# 방법 1의 첫 번째 단계: 거시경제 데이터 로드 및 파생 피처 생성
macro_df = load_and_prepare_macro(DATA_DIR, verbose=True)

print("\n생성된 거시경제 피처:")
print(macro_df.columns.tolist())

---

## Section 3. 섹터 인덱스 생성

In [ ]:
def calculate_sector_index_log(df):
    """수익률 기반 섹터 인덱스 계산 (log 변환)"""
    df_clean = df.copy()

    # 이상치 클리핑
    upper_limit = df_clean['Daily_Return'].quantile(0.99)
    lower_limit = df_clean['Daily_Return'].quantile(0.01)
    df_clean['Daily_Return'] = df_clean['Daily_Return'].clip(lower=lower_limit, upper=upper_limit)

    # 섹터/일자별 집계
    sector_daily = df_clean.groupby(['Date', 'Sector'], as_index=False).agg({
        'Daily_Return': 'mean',
        'Volume': 'sum',
        'Company': 'count'
    })
    
    sector_daily = sector_daily.sort_values(['Sector', 'Date'])
    sector_daily['Daily_Return'] = sector_daily['Daily_Return'].fillna(0)
    
    # 지수 계산
    sector_daily['Index'] = sector_daily.groupby('Sector')['Daily_Return'].transform(
        lambda x: 100 * (1 + x).cumprod()
    )
    
    # log 변환
    sector_daily['y'] = np.log(sector_daily['Index'])
    sector_daily['ds'] = sector_daily['Date']
    
    return sector_daily

sector_df = calculate_sector_index_log(df)

print(f"\n 섹터 인덱스 생성 완료: {sector_df.shape}")

---

## Section 4. Feature Engineering (개선된 버전)

In [ ]:
# 기존 피처 함수 (v1과 동일)
def build_sector_panel_from_stock_df(df, sector_index_df, key_columns, prefix="Avg_", fill_method="ffill_bfill"):
    """섹터 패널 생성 (기존 로직)"""
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])

    sector_index_df = sector_index_df.copy()
    if "Date" not in sector_index_df.columns and "ds" in sector_index_df.columns:
        sector_index_df["Date"] = pd.to_datetime(sector_index_df["ds"])
    else:
        sector_index_df["Date"] = pd.to_datetime(sector_index_df["Date"])

    # 기본 집계
    mean_cols = [c for c in key_columns if c in df.columns]
    
    if mean_cols:
        sector_feats = df.groupby(["Date", "Sector"], as_index=False)[mean_cols].mean()
        
        rename_map = {c: f"{prefix}{c}" for c in mean_cols}
        sector_feats = sector_feats.rename(columns=rename_map)
        
        sector_panel = sector_index_df.merge(sector_feats, on=["Date", "Sector"], how="left")
    else:
        sector_panel = sector_index_df.copy()

    # 정렬 + 결측 처리
    sector_panel = sector_panel.sort_values(["Sector", "ds"])

    if fill_method == "ffill_bfill":
        sector_panel = sector_panel.groupby("Sector", group_keys=False).apply(
            lambda x: x.ffill().bfill(),
            include_groups=False
        )

    return sector_panel

# 키 컬럼 정의
key_columns = [
    'Daily_Return', 'Return_1M', 'Return_3M', 'Return_6M',
    'Volatility_20d', 'RSI_14', 'BB_Width',
    'Vol_MA_20', 'Vol_Ratio', 'Vol_Z_Score',
    'Drawdown', 'MDD'
]

sector_panel = build_sector_panel_from_stock_df(
    df=df,
    sector_index_df=sector_df,
    key_columns=key_columns,
    prefix="Avg_"
)

print(f" 기본 섹터 패널 생성: {sector_panel.shape}")

###  개선된 피처 적용 (방법 1의 핵심)

In [ ]:
print("=" * 70)
print("개선된 피처 적용")
print("=" * 70)

# 1. 거시경제 데이터 병합
print("\n[1/3] 거시경제 데이터 병합")
sector_panel = merge_macro_to_sector_panel(sector_panel, macro_df, verbose=True)

# 2. 레짐 피처 추가
print("\n[2/3] 레짐 피처 추가")
sector_panel = add_regime_features(sector_panel, verbose=True)

# 3. 모멘텀 피처 추가
print("\n[3/3] 모멘텀 피처 추가")
sector_panel = add_momentum_features(sector_panel, verbose=True)

print("\n" + "=" * 70)
print(" 모든 개선 피처 적용 완료")
print("=" * 70)
print(f"최종 Shape: {sector_panel.shape}")
print(f"\n새로 추가된 컬럼:")
new_cols = [c for c in sector_panel.columns if any(x in c for x in ['DGS10', 'WTI', 'Market_Vol', 'CrossSection', 'Mom_'])]
for col in new_cols:
    print(f"  - {col}")

---

## Section 5. 휴장일 설정

In [ ]:
def get_market_holidays_simple(start_year, end_year):
    nyse_holidays = holidays.NYSE(years=range(start_year, end_year + 1))
    
    holiday_data = []
    for date, name in nyse_holidays.items():
        holiday_data.append({
            'holiday': 'market_closed',
            'ds': pd.to_datetime(date),
            'lower_window': 0,
            'upper_window': 0
        })
        
    return pd.DataFrame(holiday_data)

US_HOLIDAYS = get_market_holidays_simple(2018, 2026)
print(f"미국 시장 휴장일: {len(US_HOLIDAYS)}개")

---

## Section 6.  개선된 모델 함수 (방법 2 + 3)

### 6.1 개선된 Prophet 모델 (방법 2)

In [ ]:
def create_enhanced_prophet_model(holidays_df, sector_data=None, base_cps=0.05):
    """
    개선된 Prophet 모델 (v2)
    
    개선사항:
    - 동적 cps 조정 (섹터 변동성 기반)
    - 거시경제 regressor 추가
    - 레짐 regressor 추가
    - 모멘텀 regressor 추가
    """
    
    # 동적 cps 계산
    if sector_data is not None:
        cps = calculate_dynamic_cps(sector_data, base_cps=base_cps)
    else:
        cps = base_cps
    
    model = Prophet(
        growth='linear',
        changepoint_prior_scale=cps,  #  동적 조정
        seasonality_mode='additive',
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        holidays=holidays_df,
        interval_width=0.95
    )
    
    # 기존 기술적 지표
    tech_regressors = [
        'Avg_Volatility_20d',
        'Avg_RSI_14',
        'Avg_Vol_Z_Score',
        'Avg_BB_Width',
        'Avg_Return_3M'
    ]
    
    #  거시경제 regressor
    macro_regressors = [
        'DGS10_level',
        'DGS10_change_20d',
        'DGS10_zscore_252',
        'WTI_log',
        'WTI_mom_3M',
        'WTI_change_20d'
    ]
    
    #  레짐 regressor
    regime_regressors = [
        'Market_Vol',
        'CrossSection_Dispersion'
    ]
    
    #  모멘텀 regressor
    momentum_regressors = [
        'Mom_1M',
        'Mom_6M',
        'Mom_Accel'
    ]
    
    all_regressors = tech_regressors + macro_regressors + regime_regressors + momentum_regressors
    
    for r in all_regressors:
        model.add_regressor(r)
    
    return model, cps

print(" Enhanced Prophet 모델 함수 정의 완료")

### 6.2 개선된 XGBoost 모델 (방법 3)

In [ ]:
def train_enhanced_xgb_model(train_data, prophet_forecast, feature_cols=None, use_early_stopping=True):
    """
    개선된 XGBoost 잔차 모델 (v2)
    
    개선사항:
    - 거시경제 피처 추가
    - 레짐 피처 추가
    - 모멘텀 피처 추가
    -  Early Stopping 추가
    """
    
    # Prophet 예측값 병합
    merged = train_data.merge(
        prophet_forecast[['ds', 'Sector', 'yhat']],
        on=['ds', 'Sector'],
        how='inner'
    )
    
    # 잔차 계산
    merged['residual'] = merged['y'] - merged['yhat']
    
    #  개선된 피처 리스트 (방법 3)
    if feature_cols is None:
        feature_cols = [
            'yhat',
            
            # 기존 기술적 지표
            'Avg_Volatility_20d',
            'Avg_RSI_14',
            'Avg_Vol_Z_Score',
            'Avg_BB_Width',
            'Avg_Return_3M',
            
            #  거시경제 피처
            'DGS10_level',
            'DGS10_change_20d',
            'DGS10_zscore_252',
            'WTI_log',
            'WTI_mom_3M',
            'WTI_change_20d',
            
            #  레짐 피처
            'Market_Vol',
            'CrossSection_Dispersion',
            
            #  모멘텀 피처
            'Mom_1M',
            'Mom_6M',
            'Mom_Accel'
        ]
    
    # 실제 존재하는 컬럼만 사용
    feature_cols = [c for c in feature_cols if c in merged.columns]
    
    # 결측치 제거
    merged = merged.dropna(subset=feature_cols + ['residual'])
    
    X = merged[feature_cols].values
    y = merged['residual'].values
    
    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    #  Early Stopping 적용
    if use_early_stopping and len(X_scaled) > 100:
        # Train/Val Split (80/20)
        val_size = int(len(X_scaled) * 0.2)
        X_train, X_val = X_scaled[:-val_size], X_scaled[-val_size:]
        y_train, y_val = y[:-val_size], y[-val_size:]
        
        xgb_model = xgb.XGBRegressor(
            n_estimators=600,
            learning_rate=0.03,
            max_depth=3,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=2.0,
            min_child_weight=5,
            random_state=42,
            n_jobs=-1
        )
        
        #  Early Stopping
        xgb_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=20,
            verbose=False
        )
    else:
        # Early Stopping 없이
        xgb_model = xgb.XGBRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=3,
            random_state=42,
            n_jobs=-1
        )
        xgb_model.fit(X_scaled, y)
    
    return xgb_model, scaler, feature_cols

print(" Enhanced XGBoost 모델 함수 정의 완료")

### 6.3 통합 예측 함수

In [ ]:
def predict_sector_year_enhanced(sector_data, year, holidays_df):
    """
    개선된 년도별 예측 (Prophet v2 + XGBoost v2)
    """
    
    sectors = sector_data['Sector'].dropna().unique()
    if len(sectors) != 1:
        raise ValueError(f"sector_data must contain exactly 1 sector")
    sector_name = sectors[0]

    df = sector_data.copy()
    df['ds'] = pd.to_datetime(df['ds'])
    df = df.sort_values('ds').reset_index(drop=True)

    split_date = pd.Timestamp(f"{year}-01-01")
    next_year_date = pd.Timestamp(f"{year+1}-01-01")

    train = df[df['ds'] < split_date].copy()
    test = df[(df['ds'] >= split_date) & (df['ds'] < next_year_date)].copy()

    if len(train) == 0 or len(test) == 0:
        return None

    # 1.  Enhanced Prophet (동적 cps, 거시경제 regressor)
    prophet_model, used_cps = create_enhanced_prophet_model(holidays_df, train)
    
    # Prophet 입력 컬럼
    prophet_cols = ['ds', 'y'] + [
        'Avg_Volatility_20d', 'Avg_RSI_14', 'Avg_Vol_Z_Score', 'Avg_BB_Width', 'Avg_Return_3M',
        'DGS10_level', 'DGS10_change_20d', 'DGS10_zscore_252',
        'WTI_log', 'WTI_mom_3M', 'WTI_change_20d',
        'Market_Vol', 'CrossSection_Dispersion',
        'Mom_1M', 'Mom_6M', 'Mom_Accel'
    ]
    prophet_cols = [c for c in prophet_cols if c in train.columns]
    
    prophet_model.fit(train[prophet_cols])
    
    # Test 예측
    test_forecast_cols = [c for c in prophet_cols if c in test.columns]
    prophet_forecast = prophet_model.predict(test[test_forecast_cols])
    
    # 2.  Enhanced XGBoost (Early Stopping, 거시경제 피처)
    # Train 전체에 대해 Prophet 예측 필요
    train_prophet = prophet_model.predict(train[prophet_cols])
    train_prophet['Sector'] = sector_name
    
    xgb_model, scaler, feature_cols = train_enhanced_xgb_model(
        train,
        train_prophet,
        use_early_stopping=True
    )
    
    # Test residual 예측
    test_features = test[feature_cols].values
    test_features_scaled = scaler.transform(test_features)
    residual_predicted = xgb_model.predict(test_features_scaled)
    
    # 3. 결과 조합
    result = pd.DataFrame({
        'ds': test['ds'].values,
        'Sector': sector_name,
        'y_actual': test['y'].values,
        'yhat_prophet': prophet_forecast['yhat'].values,
        'yhat_xgb_resid': residual_predicted,
        'yhat_hybrid': prophet_forecast['yhat'].values + residual_predicted,
        'used_cps': used_cps
    })
    
    return result

print(" 통합 예측 함수 정의 완료")

---

## Section 7. Rolling Loop 실행

In [ ]:
TEST_YEARS = [2022, 2023, 2024, 2025]
sectors = sector_panel['Sector'].unique()

print("=" * 70)
print(f" Enhanced Rolling Loop v2")
print(f"Test Years: {TEST_YEARS}")
print("=" * 70)

all_predictions = []

for year in TEST_YEARS:
    print(f"\n{'='*70}")
    print(f"Year {year} 예측")
    print(f"{'='*70}")
    
    year_predictions = []
    
    for i, sector in enumerate(sectors, 1):
        print(f"  [{i}/{len(sectors)}] {sector:25s} ", end="")
        
        sector_data = sector_panel[sector_panel['Sector'] == sector].copy()
        
        try:
            pred = predict_sector_year_enhanced(sector_data, year, US_HOLIDAYS)
            
            if pred is not None and len(pred) > 0:
                pred['test_year'] = year
                pred['train_end_year'] = year - 1
                year_predictions.append(pred)
                print(f" ({len(pred)}일, cps={pred['used_cps'].iloc[0]:.3f})")
            else:
                print("  데이터 부족")
                
        except Exception as e:
            print(f" {str(e)[:30]}")
    
    if year_predictions:
        year_df = pd.concat(year_predictions, ignore_index=True)
        all_predictions.append(year_df)
        print(f"\n   {year}년 완료: {len(year_df):,} 레코드")

if all_predictions:
    df_predictions = pd.concat(all_predictions, ignore_index=True)
    print(f"\n{'='*70}")
    print(" 전체 Rolling Loop 완료")
    print(f"{'='*70}")
    print(f"총 레코드: {len(df_predictions):,}")
    print(f"섹터 수: {df_predictions['Sector'].nunique()}")
    print(f"년도: {sorted(df_predictions['test_year'].unique())}")
else:
    print("\n 예측 실패")

---

## Section 8. 평가

(평가 함수는 v1과 동일하므로 생략)

In [ ]:
def calculate_ranking_metrics(df, year, pred_col='yhat_hybrid'):
    """랭킹 기반 평가"""
    year_data = df[df['test_year'] == year].copy()
    if len(year_data) == 0:
        return None

    year_data = year_data.sort_values(['Sector', 'ds'])

    def log_return(x):
        x = x.dropna()
        if len(x) < 2:
            return np.nan
        return np.exp(x.iloc[-1] - x.iloc[0]) - 1.0

    sector_summary = (
        year_data.groupby('Sector')
        .agg(
            Actual_Return=('y_actual', log_return),
            Predicted_Return=(pred_col, log_return),
        )
        .reset_index()
        .dropna()
    )

    sector_summary['Actual_Rank'] = sector_summary['Actual_Return'].rank(ascending=False, method='min')
    sector_summary['Predicted_Rank'] = sector_summary['Predicted_Return'].rank(ascending=False, method='min')
    sector_summary['Rank_Diff'] = abs(sector_summary['Actual_Rank'] - sector_summary['Predicted_Rank'])

    spearman_corr, spearman_pvalue = spearmanr(
        sector_summary['Predicted_Return'],
        sector_summary['Actual_Return']
    )

    def top_k_hit_rate(summary_df, k):
        k = min(k, len(summary_df))
        pred_top_k = set(summary_df.nsmallest(k, 'Predicted_Rank')['Sector'])
        actual_top_k = set(summary_df.nsmallest(k, 'Actual_Rank')['Sector'])
        return len(pred_top_k & actual_top_k) / k * 100 if k > 0 else np.nan

    top3_hit = top_k_hit_rate(sector_summary, 3)
    top5_hit = top_k_hit_rate(sector_summary, 5)
    close_success_rate = (sector_summary['Rank_Diff'] <= 2).mean() * 100

    return {
        'Spearman': spearman_corr,
        'Spearman_PValue': spearman_pvalue,
        'Top3_Hit_Rate': top3_hit,
        'Top5_Hit_Rate': top5_hit,
        'Close_Success_Rate': close_success_rate,
        'Sector_Summary': sector_summary
    }

print(" 평가 함수 정의 완료")

In [ ]:
print("=" * 90)
print(" Enhanced v2 평가 결과")
print("=" * 90)

evaluation_results = []

for year in sorted(df_predictions['test_year'].unique()):
    print(f"\n{'='*90}")
    print(f"Year {year}")
    print(f"{'='*90}")
    
    ranking_metrics = calculate_ranking_metrics(df_predictions, year, 'yhat_hybrid')
    
    if ranking_metrics is None:
        print("  평가 불가")
        continue
    
    print(f"\n[핵심 랭킹 지표]")
    print(f"  Spearman: {ranking_metrics['Spearman']:.4f} (p={ranking_metrics['Spearman_PValue']:.4f})")
    print(f"  Top-3 Hit: {ranking_metrics['Top3_Hit_Rate']:.1f}%")
    print(f"  Top-5 Hit: {ranking_metrics['Top5_Hit_Rate']:.1f}%")
    print(f"  근접 성공률: {ranking_metrics['Close_Success_Rate']:.1f}%")
    
    sector_summary = ranking_metrics['Sector_Summary'].sort_values('Actual_Rank')
    print(f"\n[섹터별 순위]")
    print(f"{'Sector':25s} {'실제순위':>10s} {'예측순위':>10s} {'차이':>8s} {'실제수익률':>12s}")
    print("-" * 70)
    for _, row in sector_summary.iterrows():
        print(f"{row['Sector']:25s} {int(row['Actual_Rank']):>10d} {int(row['Predicted_Rank']):>10d} "
              f"{int(row['Rank_Diff']):>8d} {row['Actual_Return']*100:+11.2f}%")
    
    evaluation_results.append({
        'Year': year,
        'Spearman': ranking_metrics['Spearman'],
        'Top3_Hit': ranking_metrics['Top3_Hit_Rate'],
        'Top5_Hit': ranking_metrics['Top5_Hit_Rate'],
        'Close_Success': ranking_metrics['Close_Success_Rate']
    })

df_eval_summary = pd.DataFrame(evaluation_results)

print(f"\n{'='*90}")
print(" 전체 평균 성능 (v2)")
print(f"{'='*90}")
print(f"  평균 Spearman: {df_eval_summary['Spearman'].dropna().mean():.4f}")
print(f"  평균 Top-3 Hit: {df_eval_summary['Top3_Hit'].mean():.1f}%")
print(f"  평균 Top-5 Hit: {df_eval_summary['Top5_Hit'].mean():.1f}%")
print(f"  평균 근접 성공률: {df_eval_summary['Close_Success'].mean():.1f}%")

print("\n🎯 특히 2023년 성능을 확인하세요!")

---

## Section 9. 결과 저장

In [ ]:
# 결과 저장
df_predictions.to_csv(OUTPUT_DIR / 'enhanced_v2_predictions.csv', index=False)
df_eval_summary.to_csv(OUTPUT_DIR / 'enhanced_v2_evaluation.csv', index=False)

print("결과 저장 완료")
print(f"  - {OUTPUT_DIR / 'enhanced_v2_predictions.csv'}")
print(f"  - {OUTPUT_DIR / 'enhanced_v2_evaluation.csv'}")

print("\n" + "=" * 70)
print(" Enhanced Prophet v2 분석 완료")
print("=" * 70)